In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
from pprint import pprint

In [3]:
import numpy as np
import decimal
import copy

In [4]:
import time

In [5]:
import streamm

In this getting started example we will calculate the coupling between P3HT oligomers

In [6]:
import logging
logging.basicConfig(filename='p3ht_et.log',level=logging.DEBUG)

Now let's create project and resource to keep track of our work

In [7]:
p3ht_et = streamm.Project('P3HT_ET')

And a resource object to keep track of where our files are 

In [8]:
res_local = streamm.Resource('local')

Update relative location of templates directory 

In [9]:
res_local.dir['templates'] =  os.path.join(res_local.dir['home'],'..','templates','')

In [10]:
res_local.make_dir()

In [11]:
p3ht_et.set_resource(res_local)

Now we need to set a remote resource we have ssh access to and run calculations 

In [12]:
peregrine = streamm.Resource('peregrine')

In [13]:
peregrine.meta['type'] = "ssh"
peregrine.ssh['username'] = "tkemper"    
peregrine.ssh['address'] = "peregrine.hpc.nrel.gov"    
peregrine.dir['storage'] = '/mss/users/%s'%(peregrine.ssh['username'])
peregrine.dir['scratch'] = '/scratch/%s'%(peregrine.ssh['username'])
peregrine.dir['home'] = res_local.dir['home']
peregrine.dir['launch'] = res_local.dir['launch']
peregrine.dir['templates'] = res_local.dir['templates']

# Set default simulation specs
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['e-mail'] = 'travis.kemper@nrel.gov'
peregrine.properties['scratch'] = peregrine.dir['scratch']
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['walltime'] = 4
peregrine.properties['nodes'] = int(1)
peregrine.properties['ppn'] = int(24)
peregrine.properties['nproc'] = peregrine.properties['nodes']*peregrine.properties['ppn']
peregrine.properties['queue'] = 'short'
peregrine.properties['feature'] = '24core'
peregrine.properties['exe_command'] = 'qsub '

In [14]:
pprint(peregrine.properties)

{u'allocation': 'orgopv',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': '/scratch/tkemper',
 u'walltime': 4}


In [15]:
peregrine.make_dir()

Explicitely create a thiophene molecule

In [16]:
bbTh = streamm.Buildingblock('thiophene')
symbols = ['C','C','C','C','S','H','H','H','H']
positions = [ ]
positions.append([-1.55498576,-1.91131218,-0.00081000])
positions.append([-0.17775976,-1.91131218,-0.00081000])
positions.append([0.34761524,-0.57904218,-0.00081000])
positions.append([-0.65884476,0.36101082,0.00000000])
positions.append([-2.16948076,-0.35614618,-0.00000800])
positions.append([-2.18966076,-2.79526518,-0.00132100])
positions.append([0.45389024,-2.80145418,-0.00106400])
positions.append([1.41682424,-0.35961818,-0.00138200])
positions.append([-0.51943676,1.44024682,0.00064700])
for i in range(len(symbols)):
    pt_i = streamm.Particle(symbol=symbols[i])
    pos_i = positions[i]
    bbTh.add_partpos(pt_i,pos_i)

In [17]:
bbTh.particles[5].rsite = 'termcap'
bbTh.particles[6].rsite = 'funccap'
bbTh.particles[8].rsite = 'termcap'

Set some properties of the molecule to keep track of the parts

In [18]:
c_cnt =1
h_cnt =1

for pkey_i, particle_i  in bbTh.particles.iteritems():

    if( particle_i.symbol == 'C' ):
        particle_i.label = "C%d"%(c_cnt)
        particle_i.resname = "SCP2"
        particle_i.residue = 1

        c_cnt +=1 
    if( particle_i.symbol == 'S' ):
        particle_i.resname = "ThS"
        particle_i.residue = 2

    if( particle_i.symbol == 'H' ):
        particle_i.label = "H%d"%(h_cnt)
        particle_i.resname = "HA"
        particle_i.residue = 3

        h_cnt +=1 


Set the force-field type and guess some reasonable charges 

In [19]:
for pkey_i, particle_i  in bbTh.particles.iteritems():
    if( particle_i.symbol == 'C' ):
        particle_i.paramkey = 'CA'
        particle_i.charge = -0.025
    if( particle_i.symbol == 'S' ):
        particle_i.paramkey = 'S'
        particle_i.charge = -0.3
    if( particle_i.symbol == 'H' ):
        particle_i.paramkey = 'HA'
        particle_i.charge = 0.1

Check molecule is neutral 

In [20]:
total_charge = 0.0
for pkey_i, particle_i  in bbTh.particles.iteritems():
    total_charge += particle_i.charge
print total_charge

-2.77555756156e-17


Optimize structure with NWChem

But let's put it in a function this time

In [21]:
def nw_opt(project_i,bb_i,res_i):
    '''Optimize a streamm Buildingblock object with nwchem 
    
    '''
    nwchem_i = streamm.NWChem('nw_opt_{}'.format(bb_i.tag))
    print nwchem_i.tag 
    # Add thiophene structure 
    nwchem_i.strucC = copy.deepcopy(bb_i)
    # Set calculation to run on external resource
    nwchem_i.set_resource(res_i)
    # Make the local directories 
    nwchem_i.make_dir()
    #Change to the `launch` directory
    os.chdir(nwchem_i.dir['launch'])
    # Copy over templates
    nwchem_i.cp_file('templates','run',"nwchem_peregrine.pbs",'templates','launch')
    nwchem_i.cp_file('templates','nw',"nwchem.nw",'templates','launch')
    # Read in templates files 
    nwchem_i.load_str('templates','nw')        
    nwchem_i.load_str('templates','run')
    # Set calculation properties 
    nwchem_i.properties['basis'] = '6-31g'
    nwchem_i.properties['method'] = 'UHF'
    nwchem_i.properties['charge'] = 0
    nwchem_i.properties['spin_mult'] = 1
    nwchem_i.properties['task'] = 'SCF optimize'
    nwchem_i.properties['coord'] = nwchem_i.strucC.write_coord()
    # 
    pprint(nwchem_i.properties)
    # Replace <key> with properties value 
    nwchem_i.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_i.tag))
    nwchem_i.properties['input_nw'] = nwchem_i.files['input']['nw']
    nwchem_i.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_i.tag))
    #
    nwchem_i.add_file('output','log',"%s.log"%(nwchem_i.tag))
    # Save details in .json files 
    os.chdir(nwchem_i.dir['home'])
    p3ht_et.dump_json()
    # 
    os.chdir(nwchem_i.dir['launch'])
    # 
    nwchem_i.push()
    # 
    nwchem_i.run()
    # Add calculation to project
    project_i.add_calc(nwchem_i,deepcopy = True)
    # 
    return project_i 

In [22]:
p3ht_et = nw_opt(p3ht_et,bbTh,peregrine)

nw_opt_thiophene
{u'allocation': 'orgopv',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -1.55498576      -1.91131218      -0.00081000 \n     C      -0.17775976      -1.91131218      -0.00081000 \n     C       0.34761524      -0.57904218      -0.00081000 \n     C      -0.65884476       0.36101082       0.00000000 \n     S      -2.16948076      -0.35614618      -0.00000800 \n     H      -2.18966076      -2.79526518      -0.00132100 \n     H       0.45389024      -2.80145418      -0.00106400 \n     H       1.41682424      -0.35961818      -0.00138200 \n     H      -0.51943676       1.44024682       0.00064700 \n',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'finish_str': u'Total times  cpu:',
 u'maxiter': 100,
 u'method': 'UHF',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': u'/scratch/tkemper/nw_opt_thiophene/'

In [23]:
nwchem_i = p3ht_et.calculations['nw_opt_thiophene']

Check status unit finished

In [24]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()
    time.sleep(30)    

In [25]:
print nwchem_i.meta['status']

finished


In [26]:
nwchem_i.analysis()

Print energies 

In [27]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

[-0.4732391, -0.3475018, -0.3290008, 0.1318747, 0.2130991, 0.2315597, 0.2416684, 0.269473, 0.2970013, 0.3198832]
-551.190758148


Check that the positions of the structure have been optimized 

In [28]:
print bbTh.positions

[[ -1.55498576e+00  -1.91131218e+00  -8.10000000e-04]
 [ -1.77759760e-01  -1.91131218e+00  -8.10000000e-04]
 [  3.47615240e-01  -5.79042180e-01  -8.10000000e-04]
 [ -6.58844760e-01   3.61010820e-01   0.00000000e+00]
 [ -2.16948076e+00  -3.56146180e-01  -8.00000000e-06]
 [ -2.18966076e+00  -2.79526518e+00  -1.32100000e-03]
 [  4.53890240e-01  -2.80145418e+00  -1.06400000e-03]
 [  1.41682424e+00  -3.59618180e-01  -1.38200000e-03]
 [ -5.19436760e-01   1.44024682e+00   6.47000000e-04]]


In [29]:
print nwchem_i.strucC.positions

[[ -1.52897866e+00  -1.96228345e+00  -7.78590000e-04]
 [ -1.87334460e-01  -1.91419800e+00  -1.08279000e-03]
 [  3.42602010e-01  -5.70378860e-01  -6.54980000e-04]
 [ -6.05104320e-01   3.80506620e-01  -6.80400000e-05]
 [ -2.25062134e+00  -3.24145110e-01   2.59500000e-05]
 [ -2.16229521e+00  -2.82020795e+00  -9.93390000e-04]
 [  4.31149380e-01  -2.78813162e+00  -1.61256000e-03]
 [  1.39112457e+00  -3.53828660e-01  -7.51640000e-04]
 [ -4.82380800e-01   1.43977439e+00   3.58040000e-04]]


Update positions with optimized geometry 

In [30]:
for pk,p in bbTh.particles.iteritems():
    bbTh.positions[pk] = nwchem_i.strucC.positions[pk]
    print pk,p.symbol,bbTh.positions[pk]

0 C [ -1.52897866e+00  -1.96228345e+00  -7.78590000e-04]
1 C [ -1.87334460e-01  -1.91419800e+00  -1.08279000e-03]
2 C [ 0.34260201 -0.57037886 -0.00065498]
3 C [ -6.05104320e-01   3.80506620e-01  -6.80400000e-05]
4 S [ -2.25062134e+00  -3.24145110e-01   2.59500000e-05]
5 H [ -2.16229521e+00  -2.82020795e+00  -9.93390000e-04]
6 H [  4.31149380e-01  -2.78813162e+00  -1.61256000e-03]
7 H [  1.39112457e+00  -3.53828660e-01  -7.51640000e-04]
8 H [ -4.82380800e-01   1.43977439e+00   3.58040000e-04]


Store the results in a tar ball in the storage directory 

In [31]:
nwchem_i.store()

Now let us calculate the ESP charges to use in our forcefield 

Again let's make it a function

In [32]:
def nw_esp(project_i,bb_i,res_i):
    '''Calculate ESP charges of a streamm Buildingblock object with nwchem 
    
    '''
    nwchem_esp = streamm.NWChem('nw_esp_{}'.format(bb_i.tag))
    print(nwchem_esp.tag)
    # Add thiophene structure with optimized coordinates from previous calculation
    nwchem_esp.strucC = copy.deepcopy(bb_i)
    # Set calculation to run on external resource
    nwchem_esp.set_resource(res_i)
    # Add calculation to project
    project_i.add_calc(nwchem_esp)
    # Make the local directories 
    nwchem_esp.make_dir()
    # Change to the `launch` directory
    os.chdir(nwchem_esp.dir['launch'])
    #
    nwchem_esp.cp_file('templates','run',"nwchem_peregrine.pbs",'templates','launch')
    nwchem_esp.cp_file('templates','nw',"nwchem_esp.nw",'templates','launch')
    #
    nwchem_esp.load_str('templates','nw')        
    nwchem_esp.load_str('templates','run')
    # 
    nwchem_esp.properties['basis'] = '6-31g'
    nwchem_esp.properties['method'] = 'UHF'
    nwchem_esp.properties['charge'] = 0
    nwchem_esp.properties['spin_mult'] = 1
    nwchem_esp.properties['task'] = 'SCF'
    nwchem_esp.properties['coord'] = nwchem_esp.strucC.write_coord()

    pprint(nwchem_esp.properties)

    nwchem_esp.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_esp.tag))

    nwchem_esp.properties['input_nw'] = nwchem_esp.files['input']['nw']
    nwchem_esp.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_esp.tag))

    nwchem_esp.add_file('output','log',"%s.log"%(nwchem_esp.tag))

    # Save details in .json files 

    os.chdir(nwchem_esp.dir['home'])
    project_i.dump_json()

    os.chdir(nwchem_esp.dir['launch'])
    nwchem_esp.push()
    nwchem_esp.run()
    # Add calculation to project
    project_i.add_calc(nwchem_esp,deepcopy = True)
    # 
    return project_i 
    
    

In [33]:
p3ht_et = nw_esp(p3ht_et,bbTh,peregrine)

nw_esp_thiophene
{u'allocation': 'orgopv',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -1.52897866      -1.96228345      -0.00077859 \n     C      -0.18733446      -1.91419800      -0.00108279 \n     C       0.34260201      -0.57037886      -0.00065498 \n     C      -0.60510432       0.38050662      -0.00006804 \n     S      -2.25062134      -0.32414511       0.00002595 \n     H      -2.16229521      -2.82020795      -0.00099339 \n     H       0.43114938      -2.78813162      -0.00161256 \n     H       1.39112457      -0.35382866      -0.00075164 \n     H      -0.48238080       1.43977439       0.00035804 \n',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'finish_str': u'Total times  cpu:',
 u'maxiter': 100,
 u'method': 'UHF',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': u'/scratch/tkemper/nw_esp_thiophene/'

Check status until finished

In [34]:
nwchem_i = p3ht_et.calculations['nw_esp_thiophene']

In [35]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()
    time.sleep(30)    

In [36]:
p3ht_et.check()

Calculation nw_opt_thiophene has status stored
Calculation nw_esp_thiophene has status finished


In [37]:
nwchem_i.analysis()

In [38]:
total_charge = 0.0 
for pk,p in nwchem_i.strucC.particles.iteritems():
    print pk,p.symbol,p.charge
    total_charge += p.charge
print total_charge

0 C -0.025
1 C -0.025
2 C -0.025
3 C -0.025
4 S -0.3
5 H 0.1
6 H 0.1
7 H 0.1
8 H 0.1
-2.77555756156e-17


Update the charges of the Buildingblock

In [39]:
bbTh.tag += '_HFesp'

In [40]:
for pk,p in bbTh.particles.iteritems():
    p.charge = nwchem_i.strucC.particles[pk].charge
    print pk,p.symbol,p.charge

0 C -0.025
1 C -0.025
2 C -0.025
3 C -0.025
4 S -0.3
5 H 0.1
6 H 0.1
7 H 0.1
8 H 0.1


Store the results 

In [41]:
nwchem_i.store()

In [42]:
bbTh.bonded_nblist = bbTh.guess_nblist(0,radii_buffer=1.35)

In [43]:
bbTh.bonded_bonds()
bbTh.bonded_angles()
bbTh.bonded_dih()

Store a pickled object of the Buildingblock

In [44]:
os.chdir(res_local.dir['materials']) 
bbTh.dump_pickle()

Let us optimize the structure with the oplsaa force-field to check the parameters 

In [100]:
os.chdir(res_local.dir['home']) 

In [101]:
from pathlib2 import Path

In [102]:
need_files = ['oplsaa.pkl']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run forcefields.ipynb")
        os.system("jupyter nbconvert --to python  forcefields.ipynb")
        os.system("python forcefields.py")

Need to run forcefields.ipynb


In [103]:
oplsaa = streamm.forcefields.parameters.read_pickle('oplsaa')

In [105]:
print oplsaa


    Parameters 
      LJ parameters 2 
      Bond parameters 2 
      Angle parameters 2 
      Dihedral parameters 1 
      Improper Dihedral parameters 0 



In [104]:
print oplsaa.unit_conf['energy']

kCalmol


We need to add the conjugated carbons, hydrogen and sulfur atom types 

In [106]:
import streamm.forcefields.particletype as particletype

In [107]:
import pymatgen_core.core.periodic_table as periodic_table

Set some parameters from J. Am. Chem. Soc., 1996, 118 (45), pp 11225–11236

In [108]:
CA = particletype.Particletype('CA')
HA = particletype.Particletype('HA')

In [109]:
CA.update_units(oplsaa.unit_conf)
HA.update_units(oplsaa.unit_conf)

In [110]:
CA.epsilon = 0.070 # kcal/mol
CA.sigma = 3.55 # Angstroms 

In [111]:
HA.epsilon = 0.030 # kcal/mol
HA.sigma = 2.42 # Angstroms 

In [112]:
CA.mass =  periodic_table.Element['C'].atomic_mass.real
HA.mass =  periodic_table.Element['H'].atomic_mass.real

In [113]:
print CA,HA

 CA epsilon:0.07 sigma:3.55  HA epsilon:0.03 sigma:2.42


In [114]:
S = particletype.Particletype('S')

In [115]:
S.update_units(oplsaa.unit_conf)

Set some parameters from J. Am. Chem. Soc., 1996, 118 (45), pp 11225–11236

In [116]:
S.epsilon = 0.25 # kcal/mol
S.sigma = 3.55 # Angstroms 

In [117]:
S.mass =  periodic_table.Element['S'].atomic_mass.real

Add to forcefield parameters container

In [118]:
oplsaa.add_particletype(CA)
oplsaa.add_particletype(HA)
oplsaa.add_particletype(S)

Set the bond stretching parameters 

In [119]:
import streamm.forcefields.bondtype as bondtype

In [120]:
bt_i = bondtype.Bondtype('CA','HA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.080,367.0)
oplsaa.add_bondtype(bt_i)

In [121]:
bt_i = bondtype.Bondtype('CA','CA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.400,469.0)
oplsaa.add_bondtype(bt_i)

In [122]:
bt_i = bondtype.Bondtype('S','CA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.71,250.0)
oplsaa.add_bondtype(bt_i)

In [123]:
for btk,bt in oplsaa.bondtypes.iteritems():
    print btk,bt

0  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0  
1  bond  CT - CT type harmonic 
  harmonic r_0 = 1.530000 K = 268.000000 lammps index 0  gromacs index 0  
2  bond  CA - HA type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0  
3  bond  CA - CA type harmonic 
  harmonic r_0 = 1.400000 K = 469.000000 lammps index 0  gromacs index 0  
4  bond  S - CA type harmonic 
  harmonic r_0 = 1.710000 K = 250.000000 lammps index 0  gromacs index 0  


In [124]:
import streamm.forcefields.angletype as angletype

In [125]:
bat_i = angletype.Angletype('CA','CA','CA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,63.0)
oplsaa.add_angletype(bat_i)

In [126]:
bat_i = angletype.Angletype('CA','CA','HA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,35.0)
oplsaa.add_angletype(bat_i)

In [127]:
bat_i = angletype.Angletype('CA','S','CA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(92.2,70.0)
oplsaa.add_angletype(bat_i)

In [128]:
bat_i = angletype.Angletype('S','CA','HA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,35.0)
oplsaa.add_angletype(bat_i)

In [129]:
bat_i = angletype.Angletype('S','CA','CA',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(111.0,70.0)
oplsaa.add_angletype(bat_i)

In [130]:
for atk,at in oplsaa.angletypes.iteritems():
    print atk,at

0  angle  HC - CT - HC type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0  
1  angle  HC - CT - CT type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0  
2  angle  CA - CA - CA type harmonic 
  harmonic theta_0 = 120.000000 K = 63.000000 lammps index 0  gromacs index 0  
3  angle  CA - CA - HA type harmonic 
  harmonic theta_0 = 120.000000 K = 35.000000 lammps index 0  gromacs index 0  
4  angle  CA - S - CA type harmonic 
  harmonic theta_0 = 92.200000 K = 70.000000 lammps index 0  gromacs index 0  
5  angle  S - CA - HA type harmonic 
  harmonic theta_0 = 120.000000 K = 35.000000 lammps index 0  gromacs index 0  
6  angle  S - CA - CA type harmonic 
  harmonic theta_0 = 111.000000 K = 70.000000 lammps index 0  gromacs index 0  


Set some reasonable dihedral parameters

In [131]:
import streamm.forcefields.dihtype as dihtype

In [132]:
# oplsaa.dihtypes = {}

In [133]:
dih_i = dihtype.Dihtype('X','CA','CA','X',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,1.812532,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [134]:
dih_i = dihtype.Dihtype('X','S','CA','X',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,2.416710,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [135]:
dih_i = dihtype.Dihtype('S','CA','CA','HA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,1.812532,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [136]:
for dk,d in oplsaa.dihtypes.iteritems():
    print dk,d 

0  dihedral  HC - CT - CT - HC type opls 
  k1 = 0.000000 k2 = 0.000000 k3 = 0.300000 k4 = 0.000000 lammps index 0  gromcas index 0 
1  dihedral  X - CA - CA - X type opls 
  k1 = 0.000000 k2 = 1.812532 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 
2  dihedral  X - S - CA - X type opls 
  k1 = 0.000000 k2 = 2.416710 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 
3  dihedral  S - CA - CA - HA type opls 
  k1 = 0.000000 k2 = 1.812532 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 


Let us make an MD simulation of just the monomer to check that our parameters are okay

In [367]:
print peregrine.properties['nproc']

192


In [368]:
print len(p3ht_et.calculations)

12


In [369]:
def lmp_run(project_i,bb_i,param_i,res_i,md_type = 'min'):
    calc_n =  len(project_i.calculations)     
    # Create LAMMPS calculation object 
    lmp_i = streamm.LAMMPS('lmp_{}_{}_calc_{}'.format(md_type,bb_i.tag,calc_n))
    # Set parameter container 
    lmp_i.paramC = param_i
    lmp_i.set_strucC(bb_i)
    # Set force-field parameters 
    lmp_i.set_ffparam()
    # Set resource to local
    lmp_i.set_resource(res_i)
    # Make local directories
    lmp_i.make_dir()
    # Set pbc's to on
    lmp_i.strucC.lat.pbcs = [True,True,True]
    # Change to launch directory
    os.chdir(lmp_i.dir['launch'])
    # Copy over the templates from the template directory 
    lmp_i.cp_file('templates','in',"lammps_{}.in".format(md_type),'templates','launch')
    lmp_i.cp_file('templates','run',"lammps_peregrine.pbs",'templates','launch')
    # Change to scratch
    os.chdir(lmp_i.dir['launch'])
    # Read in template files and store them as strings in the `str` dictionary
    lmp_i.load_str('templates','in')
    lmp_i.load_str('templates','run')
    # Write LAMMPS .data file
    lmp_i.write_data()
    # Replace keys in template string with properties 
    lmp_i.replacewrite_prop('in','input','in','%s.in'%(lmp_i.tag))
    # Add the input file to the properties to be written into the run file
    lmp_i.properties['input_in'] = lmp_i.files['input']['in']
    lmp_i.replacewrite_prop('run','scripts','run','%s.pbs'%(lmp_i.tag))
    # Save json file in root directory
    os.chdir(lmp_i.dir['home'])
    lmp_i.dump_json()
    # Run bash script or submit to cluster
    lmp_i.add_file('output','log',"%s.log"%(lmp_i.tag))
    # Save details in .json files 
    os.chdir(lmp_i.dir['home'])
    project_i.dump_json()
    #
    os.chdir(lmp_i.dir['launch'])
    lmp_i.push()
    lmp_i.run()
    # Add calculation to project
    project_i.add_calc(lmp_i,deepcopy = True)
    # 
    return project_i     

In [138]:
p3ht_et = lmp_run(p3ht_et,bbTh,oplsaa,peregrine)

In [141]:
p3ht_et.check()

Calculation nw_opt_thiophene has status stored
Calculation nw_esp_thiophene has status stored
Calculation lmp_min_thiophene_HFesp has status finished


In [150]:
lmp_i = p3ht_et.calculations['lmp_min_thiophene_HFesp']

In [ ]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    time.sleep(30)

In [151]:
lmp_i.analysis()

>  using mdrun with len 1 
 Adding thermo keys from line: Step Temp PotEng TotEng Press Volume Lx Ly Lz E_bond E_angle E_dihed E_impro E_vdwl E_coul E_long E_tail 
 
> col  0 ['0', '0', '13.428697', '13.428697', '-0.46933067', '1000000', '100', '100', '100', '7.4350667', '3.7504985', '5.3979259e-08', '0', '-0.0096916688', '4.3386592', '-2.0858362', '0']
1 9823u  [0.0]
> col  1 ['8', '0', '5.607597', '5.607597', '0.027893444', '1000000', '100', '100', '100', '0.11529057', '3.2551753', '3.1584433e-08', '0', '-0.0095616836', '4.3330284', '-2.0863357', '0']
2 9823u  [0.0, 8.0]
 Calc 0 finished 


Energy decreased and nothing exploded so that's good

In [152]:
lmp_i.store()

Read in data file positions

In [153]:
lmp_i.pull()

Read in data file output and update positions

In [154]:
os.chdir(lmp_i.dir['launch'])

In [155]:
datafn = lmp_i.files['output']['data_1']
print datafn

min1.data


In [157]:
lmp_i.read_data_pos(datafn)

In [158]:
print lmp_i.strucC.lat.matrix

[[ 100.    0.    0.]
 [   0.  100.    0.]
 [   0.    0.  100.]]


In [159]:
lmp_i.strucC.write_xyz()

We will use the oplsaa optimized structure as the initial structure since we will be running MD 

In [160]:
bbTh.tag += '_oplsaa'

In [161]:
for pk,p in bbTh.particles.iteritems():
    bbTh.positions[pk] = lmp_i.strucC.positions[pk]
    print pk,p.symbol,bbTh.positions[pk]

0 C [ -1.56811519e+00  -1.93637672e+00  -7.65954086e-04]
1 C [ -1.72605712e-01  -1.89182626e+00  -1.07762453e-03]
2 C [ 0.33809134 -0.59678406 -0.00066163]
3 C [ -6.51388922e-01   3.88285026e-01  -5.07119165e-05]
4 S [ -2.21210590e+00  -3.39334615e-01   1.29367266e-05]
5 H [ -2.15974655e+00  -2.84091416e+00  -1.00326260e-03]
6 H [  4.46294553e-01  -2.77631869e+00  -1.60458697e-03]
7 H [  1.39411858e+00  -3.72797638e-01  -7.68173062e-04]
8 H [ -4.66380194e-01   1.45317412e+00   3.60004847e-04]


Save the Buildingblock and force-field

In [162]:
os.chdir(res_local.dir['materials']) 
bbTh.write_xyz()
bbTh.dump_pickle() 
oplsaa.dump_pickle()

Okay now that we have a handle on thiophene let's follow the same procedure for hexane

Build hexane

In [163]:
bbHex = streamm.Buildingblock('hexane')
symbols = ['C','H','H','H','C','H','H','C','H','H','C','H','H','C','H','H','C','H','H','H']
positions = [ ]
positions.append([-6.410969,-0.381641,-0.000031])
positions.append([-7.310084,0.245311,-0.000038])
positions.append([-6.456117,-1.028799,0.884636])
positions.append([-6.456111,-1.028812,-0.884689])
positions.append([-5.135268,0.467175,-0.000033])
positions.append([-5.135484,1.128782,0.877977])
positions.append([-5.135479,1.128771,-0.87805])
positions.append([-3.850566,-0.371258,-0.000024])
positions.append([-3.85112,-1.033978,0.87841])
positions.append([-3.851114,-1.033987,-0.878451])
positions.append([-2.567451,0.469603,-0.000024])
positions.append([-2.567784,1.132155,0.8784])
positions.append([-2.567776,1.132146,-0.878455])
positions.append([-1.283527,-0.370234,-0.000013])
positions.append([-1.28337,-1.032804,0.87836])
positions.append([-1.28336,-1.032812,-0.87838])
positions.append([0.00482234,0.47342231,-0.00000898])
positions.append([0.02595107,1.09220686,0.87266464])
positions.append([0.85585781,-0.17514133,0.00194589])
positions.append([0.02780957,1.08937798,-0.87463473])
for i in range(len(symbols)):
    pt_i = streamm.Particle(symbol=symbols[i])
    pos_i = positions[i]
    bbHex.add_partpos(pt_i,pos_i)

In [164]:
bbHex.particles[0].rsite = 'rg'
bbHex.particles[1].rsite = 'rgcap'

In [165]:
c_cnt =1
h_cnt =1
for pkey_i, particle_i  in bbHex.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.label = "C%d"%(c_cnt)
                particle_i.resname = "SCP3"
                particle_i.residue = c_cnt
                c_cnt +=1 
            if( particle_i.symbol == 'H' ):
                particle_i.label = "H%d"%(h_cnt)
                particle_i.resname = "HC"
                particle_i.residue = c_cnt -1 
                h_cnt +=1 

Set the parameter keys and some reasonable atomic charges 

In [166]:
for pkey_i, particle_i  in bbHex.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.paramkey = 'CT'
                particle_i.charge = -0.12

            if( particle_i.symbol == 'H' ):
                particle_i.paramkey = 'HC'
                particle_i.charge = 0.06
            print pkey_i, particle_i.symbol,particle_i.charge

0 C -0.12
1 H 0.06
2 H 0.06
3 H 0.06
4 C -0.12
5 H 0.06
6 H 0.06
7 C -0.12
8 H 0.06
9 H 0.06
10 C -0.12
11 H 0.06
12 H 0.06
13 C -0.12
14 H 0.06
15 H 0.06
16 C -0.12
17 H 0.06
18 H 0.06
19 H 0.06


In [167]:
bbHex.particles[0].charge  = -0.18
bbHex.particles[16].charge  = -0.18

Check that the molecule is neutral 

In [168]:
bbHex.calc_charge()
print bbHex.charge


0.0


Now let us optimze and calculate ESP charges for hexane

Optimize structure with NWChem

In [169]:
p3ht_et = nw_opt(p3ht_et,bbHex,peregrine)

nw_opt_hexane
{u'allocation': 'orgopv',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -6.41096900      -0.38164100      -0.00003100 \n     H      -7.31008400       0.24531100      -0.00003800 \n     H      -6.45611700      -1.02879900       0.88463600 \n     H      -6.45611100      -1.02881200      -0.88468900 \n     C      -5.13526800       0.46717500      -0.00003300 \n     H      -5.13548400       1.12878200       0.87797700 \n     H      -5.13547900       1.12877100      -0.87805000 \n     C      -3.85056600      -0.37125800      -0.00002400 \n     H      -3.85112000      -1.03397800       0.87841000 \n     H      -3.85111400      -1.03398700      -0.87845100 \n     C      -2.56745100       0.46960300      -0.00002400 \n     H      -2.56778400       1.13215500       0.87840000 \n     H      -2.56777600       1.13214600      -0.87845500 \n     C      -1.28352700      -0.37023400      -0.00001

In [170]:
nwchem_i = p3ht_et.calculations['nw_opt_hexane']

In [171]:
p3ht_et.check()

Calculation nw_opt_thiophene has status stored
Calculation nw_opt_hexane has status finished
Calculation nw_esp_thiophene has status stored
Calculation lmp_min_thiophene_HFesp has status stored


In [ ]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()
    time.sleep(30)

Get the calculation from the project object 

In [172]:
nwchem_i.analysis()

Print energies 

In [173]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

[]
-235.269664311


Check that the positions of the structure have been optimized 

In [174]:
for pk,p in bbHex.particles.iteritems():
    print pk,p.symbol,bbHex.positions[pk]

0 C [ -6.41096900e+00  -3.81641000e-01  -3.10000000e-05]
1 H [ -7.31008400e+00   2.45311000e-01  -3.80000000e-05]
2 H [-6.456117 -1.028799  0.884636]
3 H [-6.456111 -1.028812 -0.884689]
4 C [ -5.13526800e+00   4.67175000e-01  -3.30000000e-05]
5 H [-5.135484  1.128782  0.877977]
6 H [-5.135479  1.128771 -0.87805 ]
7 C [ -3.85056600e+00  -3.71258000e-01  -2.40000000e-05]
8 H [-3.85112  -1.033978  0.87841 ]
9 H [-3.851114 -1.033987 -0.878451]
10 C [ -2.56745100e+00   4.69603000e-01  -2.40000000e-05]
11 H [-2.567784  1.132155  0.8784  ]
12 H [-2.567776  1.132146 -0.878455]
13 C [ -1.28352700e+00  -3.70234000e-01  -1.30000000e-05]
14 H [-1.28337  -1.032804  0.87836 ]
15 H [-1.28336  -1.032812 -0.87838 ]
16 C [  4.82234000e-03   4.73422310e-01  -8.98000000e-06]
17 H [ 0.02595107  1.09220686  0.87266464]
18 H [ 0.85585781 -0.17514133  0.00194589]
19 H [ 0.02780957  1.08937798 -0.87463473]


In [175]:
print nwchem_i.strucC.positions

[[ -6.41096900e+00  -3.81641000e-01  -3.10000000e-05]
 [ -7.31008400e+00   2.45311000e-01  -3.80000000e-05]
 [ -6.45611700e+00  -1.02879900e+00   8.84636000e-01]
 [ -6.45611100e+00  -1.02881200e+00  -8.84689000e-01]
 [ -5.13526800e+00   4.67175000e-01  -3.30000000e-05]
 [ -5.13548400e+00   1.12878200e+00   8.77977000e-01]
 [ -5.13547900e+00   1.12877100e+00  -8.78050000e-01]
 [ -3.85056600e+00  -3.71258000e-01  -2.40000000e-05]
 [ -3.85112000e+00  -1.03397800e+00   8.78410000e-01]
 [ -3.85111400e+00  -1.03398700e+00  -8.78451000e-01]
 [ -2.56745100e+00   4.69603000e-01  -2.40000000e-05]
 [ -2.56778400e+00   1.13215500e+00   8.78400000e-01]
 [ -2.56777600e+00   1.13214600e+00  -8.78455000e-01]
 [ -1.28352700e+00  -3.70234000e-01  -1.30000000e-05]
 [ -1.28337000e+00  -1.03280400e+00   8.78360000e-01]
 [ -1.28336000e+00  -1.03281200e+00  -8.78380000e-01]
 [  4.82234000e-03   4.73422310e-01  -8.98000000e-06]
 [  2.59510700e-02   1.09220686e+00   8.72664640e-01]
 [  8.55857810e-01  -1.75141

Update positions in Buildingblock object

In [176]:
for pk,p in bbHex.particles.iteritems():
    bbHex.positions[pk] = nwchem_i.strucC.positions[pk]
    print pk,p.symbol,bbHex.positions[pk]

0 C [ -6.41096900e+00  -3.81641000e-01  -3.10000000e-05]
1 H [ -7.31008400e+00   2.45311000e-01  -3.80000000e-05]
2 H [-6.456117 -1.028799  0.884636]
3 H [-6.456111 -1.028812 -0.884689]
4 C [ -5.13526800e+00   4.67175000e-01  -3.30000000e-05]
5 H [-5.135484  1.128782  0.877977]
6 H [-5.135479  1.128771 -0.87805 ]
7 C [ -3.85056600e+00  -3.71258000e-01  -2.40000000e-05]
8 H [-3.85112  -1.033978  0.87841 ]
9 H [-3.851114 -1.033987 -0.878451]
10 C [ -2.56745100e+00   4.69603000e-01  -2.40000000e-05]
11 H [-2.567784  1.132155  0.8784  ]
12 H [-2.567776  1.132146 -0.878455]
13 C [ -1.28352700e+00  -3.70234000e-01  -1.30000000e-05]
14 H [-1.28337  -1.032804  0.87836 ]
15 H [-1.28336  -1.032812 -0.87838 ]
16 C [  4.82234000e-03   4.73422310e-01  -8.98000000e-06]
17 H [ 0.02595107  1.09220686  0.87266464]
18 H [ 0.85585781 -0.17514133  0.00194589]
19 H [ 0.02780957  1.08937798 -0.87463473]


Store the results in a tar ball in the storage directory 

In [177]:
nwchem_i.store()

Now let us calculate the ESP charges to use in our forcefield 

In [178]:
p3ht_et = nw_esp(p3ht_et,bbHex,peregrine)

nw_esp_hexane
{u'allocation': 'orgopv',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -6.41096900      -0.38164100      -0.00003100 \n     H      -7.31008400       0.24531100      -0.00003800 \n     H      -6.45611700      -1.02879900       0.88463600 \n     H      -6.45611100      -1.02881200      -0.88468900 \n     C      -5.13526800       0.46717500      -0.00003300 \n     H      -5.13548400       1.12878200       0.87797700 \n     H      -5.13547900       1.12877100      -0.87805000 \n     C      -3.85056600      -0.37125800      -0.00002400 \n     H      -3.85112000      -1.03397800       0.87841000 \n     H      -3.85111400      -1.03398700      -0.87845100 \n     C      -2.56745100       0.46960300      -0.00002400 \n     H      -2.56778400       1.13215500       0.87840000 \n     H      -2.56777600       1.13214600      -0.87845500 \n     C      -1.28352700      -0.37023400      -0.00001

Check status unit finished

In [181]:
p3ht_et.check()

Calculation nw_opt_thiophene has status stored
Calculation nw_opt_hexane has status stored
Calculation nw_esp_thiophene has status stored
Calculation lmp_min_thiophene_HFesp has status stored
Calculation nw_esp_hexane has status finished


In [179]:
nwchem_i = p3ht_et.calculations['nw_esp_hexane']

In [180]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()
    time.sleep(30)

In [182]:
nwchem_i.analysis()

In [183]:
for pk,p in nwchem_i.strucC.particles.iteritems():
    print pk,p.symbol,p.charge

0 C -0.105262
1 H 0.014539
2 H 0.012048
3 H 0.013508
4 C 0.19358
5 H -0.055571
6 H -0.050691
7 C 0.02063
8 H -0.02177
9 H -0.023452
10 C 0.038524
11 H -0.031295
12 H -0.028867
13 C 0.205973
14 H -0.055658
15 H -0.059938
16 C -0.09414
17 H 0.006364
18 H 0.00836
19 H 0.013116


In [184]:
nwchem_i.strucC.calc_charge()
print nwchem_i.strucC.charge

-2.00000000003e-06


Print energies 

In [185]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

[-0.5549626, -0.5258815, -0.5254663, -0.5002771, -0.4784797, -0.4652444, -0.4549941, -0.4542301, -0.4337884, 0.2280542]
-235.271544554


Update the charges of the Buildingblock

In [186]:
for pk,p in bbHex.particles.iteritems():
    p.charge = nwchem_i.strucC.particles[pk].charge

In [187]:
bbHex.tag += '_HFesp'

Store the results 

In [188]:
nwchem_i.store()

First we need to identify the bonding within the Buildingblock

In [189]:
bbHex.bonded_nblist = bbHex.guess_nblist(0,radii_buffer=1.35)

In [190]:
bbHex.bonded_bonds()
bbHex.bonded_angles()
bbHex.bonded_dih()

Add the need parameters the the oplsaa parameter container

In [191]:
bat_i = angletype.Angletype('CT','CT','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(109.50,40.0)
oplsaa.add_angletype(bat_i)

In [192]:
bat_i = angletype.Angletype('CT','CT','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(109.50,40.0)
oplsaa.add_angletype(bat_i)

In [193]:
bat_i = angletype.Angletype('CT','CT','HC',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(109.50,50.0)
oplsaa.add_angletype(bat_i)

In [194]:
dih_i = dihtype.Dihtype('CT','CT','CT','CT',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.433341,-0.016667,0.066668,0.0)
oplsaa.add_dihtype(dih_i)

In [195]:
dih_i = dihtype.Dihtype('HC','CT','CT','CT',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.1,0.0)
oplsaa.add_dihtype(dih_i)

In [196]:
dih_i = dihtype.Dihtype('HC','CT','CT','HC',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.1,0.0)
oplsaa.add_dihtype(dih_i)

Run a oplsaa minimization to get the minimized structure

In [197]:
p3ht_et = lmp_run(p3ht_et,bbHex,oplsaa,peregrine)

In [198]:
p3ht_et.check()

Calculation lmp_min_hexane_HFesp has status finished
Calculation nw_esp_thiophene has status stored
Calculation nw_opt_thiophene has status stored
Calculation nw_opt_hexane has status stored
Calculation nw_esp_hexane has status stored
Calculation lmp_min_thiophene_HFesp has status stored


In [199]:
lmp_i = p3ht_et.calculations['lmp_min_hexane_HFesp']

In [ ]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    time.sleep(30)

In [200]:
lmp_i.analysis()

>  using mdrun with len 1 
 Adding thermo keys from line: Step Temp PotEng TotEng Press Volume Lx Ly Lz E_bond E_angle E_dihed E_impro E_vdwl E_coul E_long E_tail 
 
> col  0 ['0', '0', '0.41190105', '0.41190105', '-1.9261181', '1000000', '100', '100', '100', '0.23947518', '2.3669071', '0.013163119', '0', '-0.77470675', '-0.33289267', '-1.100045', '0']
1 9823u  [0.0]
> col  1 ['25', '0', '-2.0815576', '-2.0815576', '-0.086547071', '1000000', '100', '100', '100', '0.0033958522', '0.09254902', '0.0004568671', '0', '-0.7104337', '-0.36748046', '-1.1000452', '0']
2 9823u  [0.0, 25.0]
 Calc 0 finished 


Energy decreased and nothing exploded so that's good

In [201]:
lmp_i.store()

Read in data file positions

In [202]:
lmp_i.pull()

Read in data file output and update positions

In [203]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

/Users/tkemper/Development/streamm-tools/examples/scratch/lmp_min_hexane_HFesp/


In [204]:
datafn = lmp_i.files['output']['data_1']
print datafn

min1.data


In [205]:
lmp_i.read_data_pos(datafn)

In [206]:
print lmp_i.strucC.lat.matrix

[[ 100.    0.    0.]
 [   0.  100.    0.]
 [   0.    0.  100.]]


In [207]:
lmp_i.strucC.write_xyz()

We will use the oplsaa optimized structure as the initial structure since we will be running MD 

In [208]:
bbHex.tag += '_oplsaa'

In [209]:
for pk,p in bbHex.particles.iteritems():
    bbHex.positions[pk] = lmp_i.strucC.positions[pk]
    print pk,p.symbol,bbHex.positions[pk]

0 C [ -6.34240754e+00  -3.82353585e-01  -5.56491659e-07]
1 H [ -7.21207849e+00   2.58183552e-01  -3.32854988e-05]
2 H [-6.34312573 -1.00185774  0.88439752]
3 H [-6.34304993 -1.0017695  -0.88457973]
4 C [ -5.08525165e+00   4.91604666e-01  -1.24122190e-05]
5 H [-5.08405992  1.1087691   0.88616587]
6 H [-5.0840334   1.10900211 -0.8857965 ]
7 C [ -3.83819213e+00  -3.96808889e-01   1.26732542e-05]
8 H [-3.84049807 -1.01380236  0.88612113]
9 H [-3.8409079  -1.01393792 -0.88631785]
10 C [ -2.58459006e+00   4.85037505e-01  -5.11126034e-04]
11 H [-2.58160377  1.10245847  0.88563837]
12 H [-2.58150702  1.10154017 -0.88697352]
13 C [ -1.33843138e+00  -4.04483016e-01   2.90122406e-04]
14 H [-1.34047321 -1.02142597  0.88636879]
15 H [-1.34008918 -1.02209928 -0.88559241]
16 C [ -8.05783703e-02   4.68424646e-01   6.94717265e-05]
17 H [-0.07976047  1.08855879  0.88419035]
18 H [ 0.78861022 -0.17281355  0.00093658]
19 H [-0.07911102  1.0872558  -0.88481048]


Save the Buildingblock and force-field

In [210]:
os.chdir(res_local.dir['materials']) 
bbHex.write_xyz()
bbHex.dump_pickle() 
oplsaa.dump_pickle()

In [211]:
print bbHex.tag,bbTh.tag

hexane_HFesp_oplsaa thiophene_HFesp_oplsaa


So let us make some P3HT oligomers 

In [212]:
os.chdir(res_local.dir['materials']) 

In [213]:
bbTh.find_rsites()
bbHex.find_rsites()

In [214]:
print(bbTh.show_rsites())

rsite:termcap[ paticle:atom[5] H1 (H) index:5 n_bonds:1] 
rsite:termcap[ paticle:atom[8] H4 (H) index:8 n_bonds:1] 
rsite:funccap[ paticle:atom[6] H2 (H) index:6 n_bonds:1] 



In [215]:
print(bbHex.show_rsites())

rsite:rg[ paticle:atom[0] C1 (C) index:0 n_bonds:4] 
rsite:rgcap[ paticle:atom[1] H1 (H) index:1 n_bonds:1] 



In [216]:
import streamm.structures.buildingblock as bb

In [217]:
ht = bb.attach(bbTh,bbHex,'funccap',0,'rgcap',0,tag='3-hexyl-thiophene')

Update bond angles and dihedrals after Buildingblock join

In [218]:
ht.bonded_bonds()
ht.bonded_angles()
ht.bonded_dih()

Check that the molecule looks good

In [219]:
ht.write_xyz()

Check the charges of the removed hydrogens got summed onto the functionalized carbons correctly

In [220]:
ht.calc_charge()
ht.charge

-1.999999999977714e-06

In [221]:
print(ht.show_rsites())

rsite:termcap[ paticle:atom[5] H1 (H) index:5 n_bonds:1] 
rsite:termcap[ paticle:atom[7] H4 (H) index:7 n_bonds:1] 
rsite:rg[ paticle:atom[8] C1 (C) index:8 n_bonds:4] 



Add inter thiophene hexane parameters

In [222]:
bt_i = bondtype.Bondtype('CT','CA',unit_conf=oplsaa.unit_conf)
bt_i.setharmonic(1.51,317.0)
oplsaa.add_bondtype(bt_i)

Bond angle parameters 

In [223]:
bat_i = angletype.Angletype('CA','CA','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,70.0)
oplsaa.add_angletype(bat_i)


bat_i = angletype.Angletype('HA','CA','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(120.0,35.0)
oplsaa.add_angletype(bat_i)



bat_i = angletype.Angletype('CA','CT','HC',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(109.5,50.0)
oplsaa.add_angletype(bat_i)

bat_i = angletype.Angletype('CA','CT','CT',unit_conf=oplsaa.unit_conf)
bat_i.setharmonic(114.0,63.0)
oplsaa.add_angletype(bat_i)

In [224]:
for atk,at in oplsaa.angletypes.iteritems():
    print atk,at

0  angle  HC - CT - HC type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0  
1  angle  HC - CT - CT type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0  
2  angle  CA - CA - CA type harmonic 
  harmonic theta_0 = 120.000000 K = 63.000000 lammps index 0  gromacs index 0  
3  angle  CA - CA - HA type harmonic 
  harmonic theta_0 = 120.000000 K = 35.000000 lammps index 0  gromacs index 0  
4  angle  CA - S - CA type harmonic 
  harmonic theta_0 = 92.200000 K = 70.000000 lammps index 0  gromacs index 0  
5  angle  S - CA - HA type harmonic 
  harmonic theta_0 = 120.000000 K = 35.000000 lammps index 0  gromacs index 0  
6  angle  S - CA - CA type harmonic 
  harmonic theta_0 = 111.000000 K = 70.000000 lammps index 0  gromacs index 0  
7  angle  CT - CT - CT type harmonic 
  harmonic theta_0 = 109.500000 K = 40.000000 lammps index 0  gromacs index 0  
8  angle  CT - CT - CT type harmonic 
  harmonic theta_0 = 1

Note: The inter-ring torsional is not consider as a seperate set of parameters for the simplicity of this example

In [225]:
dih_i = dihtype.Dihtype('HC','CT','CT','CA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.1,0.0)
oplsaa.add_dihtype(dih_i)

In [226]:
dih_i = dihtype.Dihtype('CT','CT','CT','CA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.433341,-0.016667,0.066668,0.0)
oplsaa.add_dihtype(dih_i)

In [227]:
dih_i = dihtype.Dihtype('HC','CT','CA','CA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.1,0.0)
oplsaa.add_dihtype(dih_i)

In [228]:
dih_i = dihtype.Dihtype('CT','CT','CA','CA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,-0.0,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [229]:
for dk,d in oplsaa.dihtypes.iteritems():
    print dk,d 

0  dihedral  HC - CT - CT - HC type opls 
  k1 = 0.000000 k2 = 0.000000 k3 = 0.300000 k4 = 0.000000 lammps index 0  gromcas index 0 
1  dihedral  X - CA - CA - X type opls 
  k1 = 0.000000 k2 = 1.812532 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 
2  dihedral  X - S - CA - X type opls 
  k1 = 0.000000 k2 = 2.416710 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 
3  dihedral  S - CA - CA - HA type opls 
  k1 = 0.000000 k2 = 1.812532 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 
4  dihedral  CT - CT - CT - CT type opls 
  k1 = 0.433341 k2 = -0.016667 k3 = 0.066668 k4 = 0.000000 lammps index 0  gromcas index 0 
5  dihedral  HC - CT - CT - CT type opls 
  k1 = 0.000000 k2 = -0.000000 k3 = 0.100000 k4 = 0.000000 lammps index 0  gromcas index 0 
6  dihedral  HC - CT - CT - HC type opls 
  k1 = 0.000000 k2 = -0.000000 k3 = 0.100000 k4 = 0.000000 lammps index 0  gromcas index 0 
7  dihedral  HC - CT - CT - CA type opls 
  k1 = 0.000000 k2 = -0.000000

Run a oplsaa minimization to get the minimized structure

In [230]:
p3ht_et = lmp_run(p3ht_et,ht,oplsaa,peregrine)

In [231]:
p3ht_et.check()

Calculation lmp_min_hexane_HFesp has status stored
Calculation nw_esp_thiophene has status stored
Calculation nw_opt_thiophene has status stored
Calculation lmp_min_3-hexyl-thiophene has status finished
Calculation nw_opt_hexane has status stored
Calculation nw_esp_hexane has status stored
Calculation lmp_min_thiophene_HFesp has status stored


In [232]:
lmp_i = p3ht_et.calculations['lmp_min_3-hexyl-thiophene']

In [ ]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    time.sleep(30)

In [233]:
lmp_i.analysis()

>  using mdrun with len 1 
 Adding thermo keys from line: Step Temp PotEng TotEng Press Volume Lx Ly Lz E_bond E_angle E_dihed E_impro E_vdwl E_coul E_long E_tail 
 
> col  0 ['0', '0', '23.580158', '23.580158', '6.3834374', '1000000', '100', '100', '100', '10.948137', '4.1580737', '0.0004523371', '0', '3.7401026', '7.5074743', '-2.7740824', '0']
1 9823u  [0.0]
> col  1 ['35', '0', '6.61811', '6.61811', '0.088287602', '1000000', '100', '100', '100', '0.098322403', '3.1945748', '0.0010631524', '0', '-1.2093818', '7.307451', '-2.7739196', '0']
2 9823u  [0.0, 35.0]
 Calc 0 finished 


Energy decreased and nothing exploded so that's good

In [234]:
lmp_i.store()

Read in data file positions

In [235]:
lmp_i.pull()

Read in data file output and update positions

In [236]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

/Users/tkemper/Development/streamm-tools/examples/scratch/lmp_min_3-hexyl-thiophene/


In [237]:
datafn = lmp_i.files['output']['data_1']
print datafn

min1.data


In [238]:
lmp_i.read_data_pos(datafn)

In [239]:
print lmp_i.strucC.lat.matrix

[[ 100.    0.    0.]
 [   0.  100.    0.]
 [   0.    0.  100.]]


We will use the oplsaa optimized structure as the initial structure since we will be running MD 

In [241]:
ht.tag += '_oplsaa'

In [242]:
for pk,p in ht.particles.iteritems():
    ht.positions[pk] = lmp_i.strucC.positions[pk]
    print pk,p.symbol,ht.positions[pk]

0 C [  8.19506486e-01   1.10111983e+00  -6.86476607e-04]
1 C [ 0.13609175 -0.12016286 -0.00023107]
2 C [  1.00347430e+00  -1.21177477e+00  -1.20154076e-04]
3 C [  2.35396842e+00  -8.64526955e-01  -2.45335666e-04]
4 S [  2.52295837e+00   8.45769468e-01  -1.03410286e-05]
5 H [  3.50495491e-01   2.07469182e+00   2.87510191e-04]
6 H [  6.73288023e-01  -2.24006710e+00   1.25399957e-03]
7 H [  3.17179666e+00  -1.57145729e+00   1.04534620e-03]
8 C [ -1.37450557e+00  -1.68170109e-02  -4.32546771e-04]
9 H [-1.70780472  0.49995342  0.88735419]
10 H [-1.70863118  0.50000973 -0.88787442]
11 C [ -1.93124005e+00  -1.44428553e+00  -2.73010843e-04]
12 H [-1.58522339 -1.95539447  0.8861431 ]
13 H [-1.58556737 -1.95539379 -0.88682515]
14 C [ -3.45891385e+00  -1.44292112e+00  -5.71158329e-05]
15 H [-3.81835261 -0.94584429  0.88890749]
16 H [-3.81849807 -0.94513283 -0.88855736]
17 C [ -3.92879673e+00  -2.90059160e+00  -5.75947378e-04]
18 H [-3.55631034 -3.39457241  0.884721  ]
19 H [-3.55708709 -3.3935905

Save the Buildingblock and force-field

In [243]:
os.chdir(res_local.dir['materials']) 
ht.write_xyz()
ht.dump_pickle() 
oplsaa.dump_pickle()

Okay we have the monomer, so let's make a pentamer 

In [244]:
penta_ht = copy.deepcopy(ht)

In [245]:
# penta_ht = ht.prepattach('termcap',0,dir=-1,yangle=90.0)

In [246]:
for n in range(4):
    penta_ht = bb.attach(penta_ht,ht,'termcap',1,'termcap',0,tag='penta_3-hexyl-thiophene')

Check the charges of the removed hydrogens got summed onto the functionalized carbons correctly

In [247]:
penta_ht.calc_charge()
penta_ht.charge

-9.99999999976367e-06

In [248]:
penta_ht.write_xyz()

Well it's cis, but we can run some high temperature MD to randomize that 

Update bond angles and dihedrals after Buildingblock join

In [249]:
penta_ht.bonded_bonds()
penta_ht.bonded_angles()
penta_ht.bonded_dih()

In [250]:
print penta_ht.print_properties()

 n_particles:127 
 n_bonds:131
 n_angles:245
 n_dihedrals:351
 n_impropers:0


Run a oplsaa minimization to get the minimized structure

In [251]:
p3ht_et = lmp_run(p3ht_et,penta_ht,oplsaa,peregrine)

In [252]:
p3ht_et.check()

Calculation lmp_min_penta_3-hexyl-thiophene has status finished
Calculation lmp_min_hexane_HFesp has status stored
Calculation nw_esp_thiophene has status stored
Calculation nw_opt_thiophene has status stored
Calculation lmp_min_3-hexyl-thiophene has status stored
Calculation nw_opt_hexane has status stored
Calculation nw_esp_hexane has status stored
Calculation lmp_min_thiophene_HFesp has status stored


In [253]:
lmp_i = p3ht_et.calculations['lmp_min_penta_3-hexyl-thiophene']

In [254]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    # time.sleep(30)

In [255]:
lmp_i.analysis()

>  using mdrun with len 1 
 Adding thermo keys from line: Step Temp PotEng TotEng Press Volume Lx Ly Lz E_bond E_angle E_dihed E_impro E_vdwl E_coul E_long E_tail 
 
> col  0 ['0', '0', '669.25223', '669.25223', '179.23982', '1000000', '100', '100', '100', '7.2430833', '70.753731', '1.005452', '0', '591.08212', '12.390297', '-13.222452', '0']
1 9823u  [0.0]
> col  1 ['49', '0', '62.281426', '62.281426', '4.0178179', '1000000', '100', '100', '100', '5.1515074', '51.799809', '1.0411164', '0', '4.3197426', '13.193588', '-13.224337', '0']
2 9823u  [0.0, 49.0]
 Calc 0 finished 


Energy decreased and nothing exploded so that's good

In [256]:
lmp_i.store()

Read in data file positions

In [257]:
lmp_i.pull()

Read in data file output and update positions

In [258]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

/Users/tkemper/Development/streamm-tools/examples/scratch/lmp_min_penta_3-hexyl-thiophene/


In [259]:
datafn = lmp_i.files['output']['data_1']
print datafn

min1.data


In [260]:
lmp_i.read_data_pos(datafn)

In [261]:
print lmp_i.strucC.lat.matrix

[[ 100.    0.    0.]
 [   0.  100.    0.]
 [   0.    0.  100.]]


In [262]:
lmp_i.strucC.write_xyz()

We will use the oplsaa optimized structure as the initial structure since we will be running MD 

In [263]:
penta_ht.tag += '_oplsaa'

In [264]:
for pk,p in penta_ht.particles.iteritems():
    penta_ht.positions[pk] = lmp_i.strucC.positions[pk]
    print pk,p.symbol,penta_ht.positions[pk]

0 C [ 8.62838291 -9.6148025   0.04147399]
1 C [ 9.87907038 -8.98326809  0.04300988]
2 C [ 9.78264727 -7.60620294  0.0327691 ]
3 C [ 8.4789618  -7.12412685  0.01405436]
4 S [ 7.3525195  -8.42517723  0.03645673]
5 H [  8.47112608 -10.6835391    0.04846445]
6 H [ 10.64848382  -6.98441127   0.04207178]
7 C [ 11.23395722  -9.67657399   0.0552893 ]
8 H [ 11.29854166 -10.29153458   0.94250627]
9 H [ 11.31025638 -10.29932489  -0.82554873]
10 C [ 12.42562524  -8.68781111   0.05865442]
11 H [ 12.35666551  -8.06769229   0.93993885]
12 H [ 12.36759717  -8.07477944  -0.82835803]
13 C [ 13.80369793  -9.34751239   0.06959028]
14 H [ 13.90703396  -9.94453685   0.96238491]
15 H [ 13.91735878  -9.95216696  -0.81680095]
16 C [ 14.86270025  -8.23344551   0.07096294]
17 H [ 14.74205778  -7.62010683   0.95261903]
18 H [ 14.75233548  -7.62782282  -0.81735204]
19 C [ 16.26111663  -8.85472014   0.08184265]
20 H [ 16.37286821  -9.45680317   0.97116133]
21 H [ 16.3835382   -9.46370136  -0.80135214]
22 C [ 17.334

Save the Buildingblock and force-field

In [266]:
os.chdir(res_local.dir['materials']) 
penta_ht.write_xyz()
penta_ht.dump_pickle() 
oplsaa.dump_pickle()

Cool let's run some MD

In [267]:
p3ht_et = lmp_run(p3ht_et,penta_ht,oplsaa,peregrine,md_type='nvt')

In [268]:
p3ht_et.check()

Calculation lmp_min_penta_3-hexyl-thiophene has status stored
Calculation lmp_min_hexane_HFesp has status stored
Calculation nw_esp_thiophene has status stored
Calculation nw_opt_thiophene has status stored
Calculation lmp_min_3-hexyl-thiophene has status stored
Calculation nw_opt_hexane has status stored
Calculation lmp_nvt_penta_3-hexyl-thiophene_oplsaa has status finished
Calculation nw_esp_hexane has status stored
Calculation lmp_min_thiophene_HFesp has status stored


In [269]:
lmp_i = p3ht_et.calculations['lmp_nvt_penta_3-hexyl-thiophene_oplsaa']

In [ ]:
while( lmp_i.meta['status'] != 'finished'):
    lmp_i.check()
    time.sleep(30)

In [270]:
lmp_i.analysis()

>  using mdrun with len 3 
 Adding thermo keys from line: Step Temp PotEng TotEng Press Volume Lx Ly Lz E_bond E_angle E_dihed E_impro E_vdwl E_coul E_long E_tail 
 
> col  0 ['0', '0', '62.281429', '62.281429', '4.0178077', '1000000', '100', '100', '100', '5.1515497', '51.799764', '1.0411168', '0', '4.3197491', '13.193586', '-13.224337', '0']
1 9823u  [0.0]
> col  1 ['1', '0', '62.243681', '62.243681', '4.0684785', '1000000', '100', '100', '100', '4.9851394', '51.92281', '1.0418979', '0', '4.3269484', '13.191294', '-13.224409', '0']
2 9823u  [0.0, 1.0]
 Calc 0 finished 
 Adding thermo keys from line: Step Temp PotEng TotEng Press Volume Lx Ly Lz E_bond E_angle E_dihed E_impro E_vdwl E_coul E_long E_tail 
 
> col  0 ['1', '0', '62.243681', '62.243681', '4.0684785', '1000000', '100', '100', '100', '4.9851394', '51.92281', '1.0418979', '0', '4.3269484', '13.191294', '-13.224409', '0']
1 9823u  [1.0]
> col  1 ['1000', '63.32593', '35.72754', '59.511623', '-0.48065001', '1000000', '100', '

In [271]:
lmp_i.store()

Read in data file positions

In [272]:
lmp_i.pull()

Read in data file output and update positions

In [273]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

/Users/tkemper/Development/streamm-tools/examples/scratch/lmp_nvt_penta_3-hexyl-thiophene_oplsaa/


In [276]:
datafn = lmp_i.files['output']['data_3']
print datafn

min2.data


In [277]:
lmp_i.read_data_pos(datafn)

In [278]:
print lmp_i.strucC.lat.matrix

[[ 100.    0.    0.]
 [   0.  100.    0.]
 [   0.    0.  100.]]


In [279]:
lmp_i.strucC.write_xyz()

Awesome! We have a randomized pentamer, so let's save that as new Buildingblock

In [280]:
bbPHTh_1 = copy.deepcopy(lmp_i.strucC)

In [284]:
print bbPHTh_1

 penta_3-hexyl-thiophene_oplsaa


In [314]:
print bbPHTh_1.n_particles

127


Now let's replicate the oligomer 50 times to create a low density system

In [317]:
seed = 394572

In [318]:
pHTh_x = copy.deepcopy(bbPHTh_1)
pHTh_x = streamm.add_struc(pHTh_x,bbPHTh_1,49,seed)

No overlap found adding structure 0
No overlap found adding structure 1
No overlap found adding structure 2
No overlap found adding structure 3
No overlap found adding structure 4
No overlap found adding structure 5
No overlap found adding structure 6
No overlap found adding structure 7
No overlap found adding structure 8
No overlap found adding structure 9
No overlap found adding structure 10
No overlap found adding structure 11
No overlap found adding structure 12
No overlap found adding structure 13
No overlap found adding structure 14
No overlap found adding structure 15
No overlap found adding structure 16
No overlap found adding structure 17
No overlap found adding structure 18
No overlap found adding structure 19
No overlap found adding structure 20
No overlap found adding structure 21
No overlap found adding structure 22
No overlap found adding structure 23
No overlap found adding structure 24
No overlap found adding structure 25
No overlap found adding structure 26
No overlap 

Increase the box size

In [319]:
pHTh_x.lat.matrix = [ 200.,0.,0., 0.,200.,0.,  0.,0.,200.]

In [322]:
print pHTh_x.lat.matrix

[[ 200.    0.    0.]
 [   0.  200.    0.]
 [   0.    0.  200.]]


Randomly place oligomers into the simulation cell

In [320]:
pHTh_x.tag = 'p3HTx50'

In [321]:
os.chdir(res_local.dir['materials']) 
pHTh_x.write_xyz()
# lmp_i.strucC.dump_pickle() 

Run a heat cool cycle with NPT to create a solid phase representation of p3HT

In [325]:
p3ht_et = lmp_run(p3ht_et,pHTh_x,oplsaa,peregrine,md_type = 'equ0')

In [338]:
p3ht_et.check()

Calculation lmp_min_penta_3-hexyl-thiophene has status stored
Calculation lmp_min_hexane_HFesp has status stored
Calculation nw_esp_thiophene has status stored
Calculation nw_opt_thiophene has status stored
Calculation lmp_equ0_p3HTx50 has status finished
Calculation lmp_min_3-hexyl-thiophene has status stored
Calculation nw_opt_hexane has status stored
Calculation lmp_equ1_p3HTx50 has status stored
Calculation lmp_nvt_penta_3-hexyl-thiophene_oplsaa has status stored
Calculation nw_esp_hexane has status stored
Calculation lmp_min_thiophene_HFesp has status stored


In [339]:
lmp_i = p3ht_et.calculations['lmp_equ0_p3HTx50']

In [340]:
lmp_i.analysis()

>  using mdrun with len 3 
 Adding thermo keys from line: Step Temp PotEng TotEng Press Volume Lx Ly Lz E_bond E_angle E_dihed E_impro E_vdwl E_coul E_long E_tail 
 
> col  0 ['0', '0', '877.02859', '877.02859', '-2.6925996', '8000000', '200', '200', '200', '31.479356', '1242.1515', '1733.6988', '0', '-680.37254', '-265.07061', '-1184.858', '0']
1 9823u  [0.0]
> col  1 ['1000', '12.115252', '657.28449', '886.56761', '-3.0257293', '7188102.5', '192.99146', '192.99146', '192.99146', '34.064197', '1162.8705', '1706.4011', '0', '-732.05457', '-324.43684', '-1189.5599', '0']
2 9823u  [0.0, 1000.0]
> col  2 ['2000', '17.292782', '573.85713', '901.12585', '5.3028331', '6452781.5', '186.17255', '186.17255', '186.17255', '29.158838', '1140.9248', '1698.0417', '0', '-775.27359', '-329.84463', '-1189.15', '0']
[0.0, 1000.0]
3 9823u  [0.0, 1000.0, 2000.0]
> col  3 ['3000', '18.259997', '567.66256', '913.23598', '1.3762115', '5705518.3', '178.68923', '178.68923', '178.68923', '32.250001', '995.6628

In [347]:
print len(lmp_i.properties['run_list'])

6


In [345]:
lmp_i.store()

In [346]:
lmp_i.pull()

Read in data file output and update positions

In [348]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

/Users/tkemper/Development/streamm-tools/examples/scratch/lmp_equ0_p3HTx50/


In [349]:
datafn = lmp_i.files['output']['data_3']
print datafn

equ0.data


In [350]:
lmp_i.read_data_pos(datafn)

In [351]:
print lmp_i.strucC.lat.matrix

[[ 68.63132313   0.           0.        ]
 [  0.          68.63132313   0.        ]
 [  0.           0.          68.63132313]]


In [352]:
lmp_i.strucC.write_xyz()

In [354]:
lmp_i.strucC.tag += '_equ0'

In [355]:
os.chdir(res_local.dir['materials']) 
lmp_i.strucC.write_xyz()

Cool looks like we got the box size down a bit, but we will need to run some more to get a solid system

Let's change some of our run parameters for this longer run

In [364]:
print peregrine.properties.keys()

[u'queue', 'scratch', u'ppn', u'feature', u'allocation', 'e-mail', u'exe_command', u'nproc', u'nodes', u'walltime', u'pmem']


In [370]:
peregrine.properties['queue'] = 'batch'
peregrine.properties['nodes'] =  8
peregrine.properties['ppn'] =  24
peregrine.properties['nproc'] =  8*24
peregrine.properties['walltime'] =  48

In [372]:
p3ht_et = lmp_run(p3ht_et,lmp_i.strucC,oplsaa,peregrine,md_type = 'equ1')

In [359]:
p3ht_et.check()

Calculation lmp_min_penta_3-hexyl-thiophene has status stored
Calculation lmp_min_hexane_HFesp has status stored
Calculation nw_esp_thiophene has status stored
Calculation lmp_equ1_p3HTx50_equ0 has status written
Calculation nw_opt_thiophene has status stored
Calculation lmp_equ0_p3HTx50 has status stored
Calculation lmp_min_3-hexyl-thiophene has status stored
Calculation nw_opt_hexane has status stored
Calculation lmp_equ1_p3HTx50 has status stored
Calculation lmp_nvt_penta_3-hexyl-thiophene_oplsaa has status stored
Calculation nw_esp_hexane has status stored
Calculation lmp_min_thiophene_HFesp has status stored


In [358]:
lmp_i = p3ht_et.calculations['lmp_equ1_p3HTx50_equ0'] 

In [340]:
lmp_i.analysis()

>  using mdrun with len 3 
 Adding thermo keys from line: Step Temp PotEng TotEng Press Volume Lx Ly Lz E_bond E_angle E_dihed E_impro E_vdwl E_coul E_long E_tail 
 
> col  0 ['0', '0', '877.02859', '877.02859', '-2.6925996', '8000000', '200', '200', '200', '31.479356', '1242.1515', '1733.6988', '0', '-680.37254', '-265.07061', '-1184.858', '0']
1 9823u  [0.0]
> col  1 ['1000', '12.115252', '657.28449', '886.56761', '-3.0257293', '7188102.5', '192.99146', '192.99146', '192.99146', '34.064197', '1162.8705', '1706.4011', '0', '-732.05457', '-324.43684', '-1189.5599', '0']
2 9823u  [0.0, 1000.0]
> col  2 ['2000', '17.292782', '573.85713', '901.12585', '5.3028331', '6452781.5', '186.17255', '186.17255', '186.17255', '29.158838', '1140.9248', '1698.0417', '0', '-775.27359', '-329.84463', '-1189.15', '0']
[0.0, 1000.0]
3 9823u  [0.0, 1000.0, 2000.0]
> col  3 ['3000', '18.259997', '567.66256', '913.23598', '1.3762115', '5705518.3', '178.68923', '178.68923', '178.68923', '32.250001', '995.6628

In [347]:
print len(lmp_i.properties['run_list'])

6


In [345]:
lmp_i.store()

In [346]:
lmp_i.pull()

Read in data file output and update positions

In [348]:
print lmp_i.dir['launch']
os.chdir(lmp_i.dir['launch'])

/Users/tkemper/Development/streamm-tools/examples/scratch/lmp_equ0_p3HTx50/


In [349]:
datafn = lmp_i.files['output']['data_3']
print datafn

equ0.data


In [350]:
lmp_i.read_data_pos(datafn)

In [351]:
print lmp_i.strucC.lat.matrix

[[ 68.63132313   0.           0.        ]
 [  0.          68.63132313   0.        ]
 [  0.           0.          68.63132313]]


In [352]:
lmp_i.strucC.write_xyz()

Sweet as bro!